In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
import os
import math
from os.path import join
from pathlib import Path
import pandas as pd
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
from shapely.geometry import shape

In [3]:
# Filepaths

# Get the absolute path to the precal_hazard directory
# Which is two directories above notebooks/exploration/
abs_dir = os.path.abspath(Path(os.getcwd()).parents[1])
# Get raw data directory
fr = join(abs_dir, 'data', 'raw')
# Get interim data directory
fi = join(abs_dir, 'data', 'interim')
# Get processed data directory
fp = join(abs_dir, 'data', 'processed')

# Get National Structure Inventory Data

In [4]:
# Call the NSI API by fips
# Camden, County NJ
fips_list = ['34007']

# Get the URL
url = "https://nsi.sec.usace.army.mil/nsiapi/structures"

# Loop through counties, 
# Get the data from the NSI API
# Store in dataframe
# Add to list
# Concat all the dfs

# List for NSI DFs
nsi_df_list = []

for fips in fips_list:
    # GET Request
    nsi_get = requests.get(url + '?fips=' + fips)
    
    # Temp data frame
    temp = pd.json_normalize(nsi_get.json()['features'])
    
    # Add to list
    nsi_df_list.append(temp)

# Concat
nsi = pd.concat(nsi_df_list, axis=0)

# Write to file
nsi.to_parquet(join(fr, 'exposure', 'nsi.pqt'))

# Download NFIP Data

In [51]:
# pol Policies for Camden
# Call the pol API by fips
# Camden, County NJ
fips_list = ['34007']

# Get the URL for querying policies
url = "https://www.fema.gov/api/open/v1/FimaNfipPolicies?$"
# Get the URL for # policies that meet request
check = url + "inlinecount=allpages&$top=1&$select=id&$"


# Loop through counties, 
# Get the data from the Pols API
# Store in dataframe
# Add to list
# Concat all the dfs

# List for Pols DFs
pol_df_list = []

# NFIP API usage adapts R code here: https://docs.ropensci.org/rfema/
# And follows OpenFEMA guide: 
# https://www.fema.gov/about/openfema/working-with-large-data-sets#app-a

for fips in fips_list:
    # County endpoint
    c_end = "filter=countyCode%20eq%20%27" + fips + "%27"
    
    # First, get the total number of records
    records = requests.get(check + c_end)
    n_rec = pd.json_normalize(records.json())['metadata.count'][0]
    
    # Get iterations needed (1,000 record limit)
    iterations = math.ceil(n_rec / 1000)
    
    # Now, download 1,000 records at a time and store in list
    # Loop through required iterations and keep appending policy 
    # data from the GET request to the pol_df_list
    for i in range(iterations):
        skip_str = "&$skip=" + str(i*1000)
    
        # GET Request
        pol_get = requests.get(url + c_end + skip_str)

        # Temp data frame
        temp = pd.json_normalize(pol_get.json()['FimaNfipPolicies'])

        # Add to list
        pol_df_list.append(temp)

# Concat
nfip_pol = pd.concat(pol_df_list, axis=0)

# Write to file
nfip_pol.to_parquet(join(fr, 'exposure', 'nfip_pols.pqt'))

In [62]:
# NFIP Claims for Camden
# claim claimicies for Camden
# Call the claim API by fips
# Camden, County NJ
fips_list = ['34007']

# Get the URL for querying claimicies
url = "https://www.fema.gov/api/open/v1/FimaNfipClaims?$"
# Get the URL for # claimicies that meet request
check = url + "inlinecount=allpages&$top=1&$select=id&$"


# Loop through counties, 
# Get the data from the claims API
# Store in dataframe
# Add to list
# Concat all the dfs

# List for claims DFs
claim_df_list = []

# NFIP API usage adapts R code here: https://docs.ropensci.org/rfema/
# And follows OpenFEMA guide: 
# https://www.fema.gov/about/openfema/working-with-large-data-sets#app-a

for fips in fips_list:
    # County endpoint
    c_end = "filter=countyCode%20eq%20%27" + fips + "%27"
    
    # First, get the total number of records
    records = requests.get(check + c_end)
    n_rec = pd.json_normalize(records.json())['metadata.count'][0]
    
    # Get iterations needed (1,000 record limit)
    iterations = math.ceil(n_rec / 1000)
    
    # Now, download 1,000 records at a time and store in list
    # Loop through required iterations and keep appending claimicy 
    # data from the GET request to the claim_df_list
    for i in range(iterations):
        skip_str = "&$skip=" + str(i*1000)
    
        # GET Request
        claim_get = requests.get(url + c_end + skip_str)

        # Temp data frame
        temp = pd.json_normalize(claim_get.json()['FimaNfipClaims'])

        # Add to list
        claim_df_list.append(temp)

# Concat
nfip_claim = pd.concat(claim_df_list, axis=0)

# Write to file
nfip_claim.to_parquet(join(fr, 'exposure', 'nfip_claims.pqt'))

In [ ]:
# TODO: Add all data sources currently in keller-lab/data
# Remove those data sources from that repo, makes sense
# to instead use project by project data model
# since we need the data accessible for reproducibility
# HMGP, IHP, PA
# Also Camden County NFHL

# Download Camden County Data

In [156]:
# Parcels
par_df_list = []

# Data is in epsg: 3424
EPSG = '3424'

# Store base URL
par_url = "https://services3.arcgis.com/JGF6qCAQFbROcocK/arcgis/rest/services/Parcel_Data_2021_Redacted/FeatureServer/1/query?outFields=*"
close_str = "&f=geojson"

# Filter on municipality
mun_str = "&where=MUNICIPALITY%3D%27GLOUCESTER+CITY%27"
# Record count, 2000 at a time
rec_str = "&resultRecordCount=2000"
# Update resultOffset by 2000 at a time
rec_n = 2000

# Get number of records
num_rec_url = "https://services3.arcgis.com/JGF6qCAQFbROcocK/arcgis/rest/services/Parcel_Data_2021_Redacted/FeatureServer/1/query?where=MUNICIPALITY%3D%27GLOUCESTER+CITY%27&returnCountOnly=true&f=json"
num_r = requests.get(num_rec_url).json()['count']

# Get iterations needed 
iterations = math.ceil(num_r / rec_n)

# Now, download 2,000 records at a time and store in list
# Loop through required iterations and keep appending claimicy 
# data from the GET request to the claim_df_list
for i in range(iterations):
    skip_str = "&resultOffset=" + str(i*rec_n)

    # GET Request
    par_get = requests.get(par_url + mun_str + skip_str + rec_str + close_str)

    # Temp data frame
    temp = par_get.json()['features']
    temp_df = pd.json_normalize(temp)
    temp_geo = [shape(i['geometry']) for i in temp]

    # Geodataframe with temp_df & temp_geo linked
    par_geo = gpd.GeoDataFrame(temp_df,
                               crs=EPSG,
                               geometry=temp_geo) 

    # Add to list
    par_df_list.append(par_geo)

# Concat
pars = pd.concat(par_df_list, axis=0)

# Get back to geodataframe
pars_geo = gpd.GeoDataFrame(pars,
                            crs=EPSG,
                            geometry=pars['geometry'])

# Drop type, id, geometry.type, geometry.coordinates
drop_col = ['type', 'id', 'geometry.type', 'geometry.coordinates']
pars_geo = pars_geo.drop(columns=drop_col)

# Write data to file
pars_geo.to_file(join(fr, 'exposure', 'pc.gpkg'),
                 driver='GPKG')

ValueError: Invalid field type <class 'list'>

In [37]:
# Tracts
# tract Codes
url = "https://services3.arcgis.com/JGF6qCAQFbROcocK/arcgis/rest/services/CensusTracts/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"

# Data in epsg: 3424
EPSG = '4326'

# GET Request
tract_get = requests.get(url)

# No loop needed, just the one tracticipality
temp = tract_get.json()['features']
temp_df = pd.json_normalize(temp)
temp_geo = [shape(i['geometry']) for i in temp]

# Final df
tract_geo = gpd.GeoDataFrame(temp_df,
                           crs=EPSG,
                           geometry=temp_geo)

# Drop type, id, geometry.type, geometry.coordinates
drop_col = ['type', 'id', 'geometry.type', 'geometry.coordinates']
tract_geo = tract_geo.drop(columns=drop_col)

# Reproject to 3424
OUT_EPSG = '3424'
tract_geo = tract_geo.to_crs(epsg=OUT_EPSG)

# Write data to file
tract_geo.to_file(join(fr, 'ref', 'tracts.gpkg'),
                driver='GPKG')

In [33]:
# Zip Codes
url = "https://services3.arcgis.com/JGF6qCAQFbROcocK/arcgis/rest/services/Zip_Codes/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"

# Data in epsg: 3424
EPSG = '4326'

# GET Request
zip_get = requests.get(url)

# No loop needed, just the one zipicipality
temp = zip_get.json()['features']
temp_df = pd.json_normalize(temp)
temp_geo = [shape(i['geometry']) for i in temp]

# Final df
zip_geo = gpd.GeoDataFrame(temp_df,
                           crs=EPSG,
                           geometry=temp_geo)

# Drop type, id, geometry.type, geometry.coordinates
drop_col = ['type', 'id', 'geometry.type', 'geometry.coordinates']
zip_geo = zip_geo.drop(columns=drop_col)

# Reproject to 3424
OUT_EPSG = '3424'
zip_geo = zip_geo.to_crs(epsg=OUT_EPSG)

# Write data to file
zip_geo.to_file(join(fr, 'ref', 'zipcodes.gpkg'),
                driver='GPKG')

In [22]:
# Municipalities for Camden (useful for clipping other data to GC)
url = "https://services3.arcgis.com/JGF6qCAQFbROcocK/arcgis/rest/services/CamdenCountyMunicipalLayer/FeatureServer/0/query?f=geojson&where=(NAMELSAD%20IN%20(%27Gloucester%20City%20city%27))&outFields=*"

# Data is in epsg: 4326
# Metadata suggests 26918 but
# local download shows epsg is 4326
EPSG = '4326'

# GET Request
mun_get = requests.get(url)

# No loop needed, just the one municipality
temp = mun_get.json()['features']
temp_df = pd.json_normalize(temp)
temp_geo = [shape(i['geometry']) for i in temp]

# Final df
mun_geo = gpd.GeoDataFrame(temp_df,
                           crs=EPSG,
                           geometry=temp_geo)

# Drop type, id, geometry.type, geometry.coordinates
drop_col = ['type', 'id', 'geometry.type', 'geometry.coordinates']
mun_geo = mun_geo.drop(columns=drop_col)

# Reproject to 3424
OUT_EPSG = '3424'
mun_geo = mun_geo.to_crs(epsg=OUT_EPSG)

# Write data to file
mun_geo.to_file(join(fr, 'ref', 'city_clip.gpkg'),
                driver='GPKG')

In [208]:
# Land Uses
url = "https://services3.arcgis.com/JGF6qCAQFbROcocK/arcgis/rest/services/DVRPC_2010_Land_Use/FeatureServer/0/query?f=geojson&where=(Mun_Name%20IN%20(%27Gloucester%20City%27))&outFields=*"
# Data is in epsg: 3424
EPSG = '3424'

# GET Request
lu_get = requests.get(url)

# No loop needed because only ~100 records
# Temp data frame
temp = lu_get.json()['features']
temp_df = pd.json_normalize(temp)
temp_geo = [shape(i['geometry']) for i in temp]

# Final df
lu_geo = gpd.GeoDataFrame(temp_df,
                          crs=EPSG,
                          geometry=temp_geo)
   

# Drop type, id, geometry.type, geometry.coordinates
drop_col = ['type', 'id', 'geometry.type', 'geometry.coordinates']
lu_geo = lu_geo.drop(columns=drop_col)

# Write data to file
lu_geo.to_file(join(fr, 'exposure', 'landuse.gpkg'),
               driver='GPKG')

# Download Social Vulnerability Data

In [8]:
# NOAA SOVI
url = 'https://coast.noaa.gov/htdata/SocioEconomic/SoVI2010/SoVI_2010_NJ.zip'
save_path = join(fr, 'vulnerability', 'social', 'noaa.zip')

# Request and writing zip from here
# https://stackoverflow.com/questions/9419162/download-returned-zip-file-from-url
chunk_size = 128
r = requests.get(url, stream=True)
with open(save_path, 'wb') as fd:
    for chunk in r.iter_content(chunk_size=chunk_size):
        fd.write(chunk)

In [30]:
# NJ env. burd communities
url = "https://services1.arcgis.com/QWdNfRs7lkPq4g4Q/ArcGIS/rest/services/Overburdened_Communities_2020_Hosted/FeatureServer/0/query?where=NAME%3D%27Gloucester+City%27&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*&returnGeometry=true&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="
# Data is supposed to be in epsg: 3424
# metadata says so
# but it's in epsg 4326
EPSG = '4326'

# GET Request
ob_get = requests.get(url)

# No loop needed because only ~100 records
# Temp data frame
temp = ob_get.json()['features']
temp_df = pd.json_normalize(temp)
temp_geo = [shape(i['geometry']) for i in temp]

# Final df
ob_geo = gpd.GeoDataFrame(temp_df,
                          crs=EPSG,
                          geometry=temp_geo)
   

# Drop type, id, geometry.type, geometry.coordinates
drop_col = ['type', 'id', 'geometry.type', 'geometry.coordinates']
ob_geo = ob_geo.drop(columns=drop_col)

# Reproject to 3424
OUT_EPSG = '3424'
ob_geo = ob_geo.to_crs(epsg=OUT_EPSG)

# Write data to file
ob_geo.to_file(join(fr, 'vulnerability', 'social', 'overburdened.gpkg'),
               driver='GPKG')

In [7]:
# CEJST (download later)